In [ ]:
import sys
import os

project_root = os.path.abspath("..")
sys.path.append(project_root)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from config import *
from Data.raw_data_loader import load_raw_crypto_csv
from Models.zscore import zscore_signal
from Models.ou import ou_signal_on_spread
from Models.trend import trend_signal
from Backtest.engine import run_backtest
from Backtest.metrics import build_summary_table, rolling_sharpe

plt.style.use("default")


In [ ]:
data_path = DATA_PATH if os.path.exists(DATA_PATH) else os.path.join(project_root, "Data", "cleaned_crypto_data.csv")
df = load_raw_crypto_csv(data_path)

df = df[(df["ETH-USD_close"] > 0)]
df = df.loc["2017-11-12":].copy()

eth_ret_raw = df["ETH-USD_close"].pct_change()
ret_clip_threshold = min(1.0, float(eth_ret_raw.abs().quantile(0.999)))
df = df[eth_ret_raw.abs() < ret_clip_threshold].copy()  # trim extreme data errors driving unrealistic vol

eth_ret = df["ETH-USD_close"].pct_change().dropna()
print("Shape:", df.shape)
print("ETH Annualized Vol (cleaned):", eth_ret.std() * np.sqrt(365))
print("Daily return clip threshold:", ret_clip_threshold)


In [ ]:
z_pos = zscore_signal(
    df,
    price_column=PRICE_COLUMN_ETH,
    resid_window=ZSCORE_WINDOW,
    entry_z=ZSCORE_ENTRY_Z,
    exit_z=ZSCORE_EXIT_Z,
    long_short=ZSCORE_LONG_SHORT,
    use_vol_target=USE_VOL_TARGET,
    vol_target=VOL_TARGET,
    vol_window=VOL_WINDOW,
    max_leverage=LEVERAGE_CAP,
)


In [ ]:
z_results = run_backtest(
    price_series=df["ETH-USD_close"],
    position=z_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    annual_borrow_rate=ANNUAL_BORROW_RATE,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:
eth_log = np.log(df["ETH-USD_close"].astype(float))
eth_spread = eth_log - eth_log.rolling(OU_WINDOW, min_periods=max(30, OU_WINDOW // 3)).mean()

ou_pos = ou_signal_on_spread(
    eth_spread,
    window=OU_WINDOW,
    entry_z=OU_ENTRY_Z,
    exit_z=OU_EXIT_Z,
)

if not OU_LONG_SHORT:
    ou_pos = ou_pos.clip(lower=0.0)


In [ ]:
ou_results = run_backtest(
    price_series=df["ETH-USD_close"],
    position=ou_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    annual_borrow_rate=ANNUAL_BORROW_RATE,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:
trend_pos = trend_signal(
    df,
    price_column=PRICE_COLUMN_ETH,
    fast_window=TREND_FAST_WINDOW,
    slow_window=TREND_SLOW_WINDOW,
    long_only=TREND_LONG_ONLY,
    leverage_aggressive=TREND_AGGRESSIVE,
    leverage_neutral=TREND_NEUTRAL,
    leverage_defensive=TREND_DEFENSIVE,
)


In [ ]:
trend_results = run_backtest(
    price_series=df["ETH-USD_close"],
    position=trend_pos,
    fee_bps=FEE_BPS,
    slippage_bps=SLIPPAGE_BPS,
    annual_borrow_rate=ANNUAL_BORROW_RATE,
    leverage_cap=LEVERAGE_CAP,
)


In [ ]:
summary = build_summary_table({
    "Z-Score": {
        "returns": z_results["net_returns"],
        "position": z_pos,
    },
    "OU": {
        "returns": ou_results["net_returns"],
        "position": ou_pos,
    },
    "Trend": {
        "returns": trend_results["net_returns"],
        "position": trend_pos,
    },
})

summary


In [ ]:
plt.figure(figsize=(12,5))
plt.plot(z_results["net_equity"], label="Z-Score")
plt.plot(ou_results["net_equity"], label="OU")
plt.plot(trend_results["net_equity"], label="Trend")
plt.title("ETH Net Equity Curves")
plt.legend()
plt.show()


In [ ]:
z_gross = z_results["gross_equity"]
z_net = z_results["net_equity"]

plt.figure(figsize=(12,5))
plt.plot(z_gross, label="Z Gross")
plt.plot(z_net, label="Z Net")
plt.title("ETH Z-Score Gross vs Net Equity")
plt.legend()
plt.show()


In [ ]:
plt.figure(figsize=(12,5))
plt.plot(rolling_sharpe(z_results["net_returns"], window=365), label="Z-Score")
plt.plot(rolling_sharpe(ou_results["net_returns"], window=365), label="OU")
plt.plot(rolling_sharpe(trend_results["net_returns"], window=365), label="Trend")
plt.title("ETH Rolling 365-Day Sharpe (Net)")
plt.legend()
plt.show()


In [ ]:
def drawdown(equity: pd.Series) -> pd.Series:
    peak = equity.cummax()
    return equity / peak - 1.0

plt.figure(figsize=(12,5))
plt.plot(drawdown(z_results["net_equity"]), label="Z-Score")
plt.plot(drawdown(ou_results["net_equity"]), label="OU")
plt.plot(drawdown(trend_results["net_equity"]), label="Trend")
plt.title("ETH Drawdowns (Net)")
plt.legend()
plt.show()


In [ ]:
def annual_turnover(pos: pd.Series) -> float:
    return float(pos.diff().abs().sum() / len(pos) * 365)

print("Annual Turnover Z:", annual_turnover(z_pos))
print("Annual Turnover OU:", annual_turnover(ou_pos))
print("Annual Turnover Trend:", annual_turnover(trend_pos))


In [ ]:
def backtest_with_costs(pos, fee_bps, slip_bps):
    return run_backtest(
        price_series=df["ETH-USD_close"],
        position=pos,
        fee_bps=fee_bps,
        slippage_bps=slip_bps,
        annual_borrow_rate=ANNUAL_BORROW_RATE,
        leverage_cap=LEVERAGE_CAP,
    )["net_returns"]

cost_cases = {
    "0 bps": (0.0, 0.0),
    "10 bps": (10.0, 0.0),
    "20 bps": (10.0, 10.0),
}

rows = []
for name, (fee, slip) in cost_cases.items():
    r = backtest_with_costs(z_pos, fee, slip)
    rows.append({
        "CostCase": name,
        **build_summary_table({"tmp": {"returns": r, "position": z_pos}}).iloc[0].to_dict(),
    })

pd.DataFrame(rows).set_index("CostCase")
